In [14]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [15]:
from public_data.models import Region
from django.contrib.gis.db.models.functions import Centroid
from django.contrib.gis.db.models import Union, Extent

qs = Region.objects.aggregate(
    mpoly_union=Union("mpoly"),
    center=Centroid(Union("mpoly")),
    extent=Extent("mpoly")
)

qs

{'mpoly_union': <MultiPolygon object at 0x7f039be5b910>,
 'center': <Point object at 0x7f039be5bc10>,
 'extent': (-5.1413332610739,
  41.3336317564724,
  9.55996004666831,
  51.0890003105061)}

In [16]:
center = (qs["center"].y, qs["center"].x)
center

(46.55800750585194, 2.54985335515777)

In [17]:
coords = (qs['extent'][1], qs['extent'][0], qs['extent'][3], qs['extent'][2])
coords

(41.3336317564724, -5.1413332610739, 51.0890003105061, 9.55996004666831)

In [18]:
x = (qs['extent'][0] + qs['extent'][2]) / 2
y = (qs['extent'][1] + qs['extent'][3]) / 2
center2 = (y, x)
center2

(46.211316033489254, 2.2093133927972053)

In [19]:
from ipyleaflet import Map, Marker, GeoJSON, Polygon as iPolygon, Icon, WKTLayer
from public_data.api.serializers import RegionSerializer
from public_data.models import Region
from django.contrib.gis.geos import Polygon as dPolygon


m = Map(center=center, zoom=5)

m.add_layer(
    iPolygon(
        locations=dPolygon.from_bbox(coords).coords,
        fillColor="red",
        fillOpacity=0.1,
    )
)

m.add_layer(Marker(location=center2, draggable=True))

m

Map(center=[46.55800750585194, 2.54985335515777], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

In [27]:
from django.db.models import Func

class InnerCentroide(Func):
    """Return Centroid of maximum inner bounder circle"""
    function = 'ST_MaximumInscribedCircle'
    template = "%(function)s(%(expressions)s)"
    
Region.objects.aggregate(InnerCentroide("mpoly"))